In [3]:
import pandas as pd
import json

In [4]:
with open('chessbuds_messages.json') as j:
    data = json.load(j)

This is where I imported pandas and json, as well as the messy data. 

In [5]:
type(data)

dict

In [6]:
data.keys()

dict_keys(['participants', 'messages', 'title', 'is_still_participant', 'thread_type', 'thread_path', 'magic_words', 'joinable_mode'])

In [7]:
type(data['participants'])

list

In [8]:
data['participants'][0].keys()

dict_keys(['name'])

My first interactions with the data was to feel out which parameters had which information, and moreover, what I needed to input into pd.DataFrame. Initially, I started with participants as it was the first one, but it's dict was too simple for the database, so I moved on. 

In [9]:
type(data['messages'])

list

In [10]:
len(data['messages'])

223

In [11]:
data['messages'][0].keys()

dict_keys(['sender_name', 'timestamp_ms', 'content', 'reactions', 'type', 'is_unsent', 'is_taken_down', 'bumped_message_metadata'])

In [12]:
chessDf = pd.DataFrame(data['messages'])

Here I found that the messages was what I was going to input into the DataFrame, by checking how many keys it had. I assigned that information to the chessDf using pd.DataFrame. Below, I called it and began to examine the data and figure out what needed to be changed to make it tidy. 

In [13]:
chessDf.head()

,sender_name,timestamp_ms,content,reactions,type,is_unsent,is_taken_down,bumped_message_metadata,share,photos,gifs,users
0,Joanna Rusch,1666374933946,Maybe he just wants to ride the publicity for ...,"[{'reaction': 'ð', 'actor': 'Chad Larson'},...",Generic,False,False,{'bumped_message': 'Maybe he just wants to rid...,NaN,NaN,NaN,NaN
1,Chad Larson,1666373448613,To be fair to Hans....no one wants to be assoc...,"[{'reaction': 'ð', 'actor': 'Scott Pence'},...",Generic,False,False,{'bumped_message': 'To be fair to Hans....no o...,NaN,NaN,NaN,NaN
2,Chad Larson,1666373216381,He would have to prove he didn't cheat and tha...,"[{'reaction': 'ð', 'actor': 'Scott Pence'},...",Generic,False,False,{'bumped_message': 'He would have to prove he ...,NaN,NaN,NaN,NaN
3,Scott Pence,1666373164883,"Yeah, no way. You over shoot and hope to get a...","[{'reaction': 'ð', 'actor': 'Chad Larson'},...",Generic,False,False,"{'bumped_message': 'Yeah, no way. You over sho...",NaN,NaN,NaN,NaN
4,Chad Larson,1666373111157,"From what I see, I don't think he could win. ...",NaN,Generic,False,False,"{'bumped_message': 'From what I see, I don't t...",NaN,NaN,NaN,NaN


After viewing the head of the database, a few things stuck out to me. All of the NaN entries in the share, photos, gifs, and user collumns go againt tidy principles, because you cannot have empty values. I needed to find a way to fix this. I also thought that the reactions and bumped_message_metadata collumns needed to be cleaned up as much of the content was repeated/unreadable, simillar to when we were working on the Spotify data. 

In [14]:
type(chessDf['reactions'].iloc[0])

list

In [15]:
type(chessDf['reactions'].iloc[0][0])

dict

In [16]:
list(chessDf['reactions'])

[[{'reaction': 'ð\x9f\x91\x8d', 'actor': 'Chad Larson'},
  {'reaction': 'ð\x9f\x91\x8d', 'actor': 'Chad Larson'}],
 [{'reaction': 'ð\x9f\x98\x86', 'actor': 'Scott Pence'},
  {'reaction': 'ð\x9f\x98\x86', 'actor': 'Scott Pence'}],
 [{'reaction': 'ð\x9f\x91\x8d', 'actor': 'Scott Pence'},
  {'reaction': 'ð\x9f\x91\x8d', 'actor': 'Scott Pence'}],
 [{'reaction': 'ð\x9f\x91\x8d', 'actor': 'Chad Larson'},
  {'reaction': 'ð\x9f\x91\x8d', 'actor': 'Chad Larson'}],
 nan,
 [{'reaction': 'ð\x9f\x91\x8d', 'actor': 'Chad Larson'},
  {'reaction': 'ð\x9f\x91\x8d', 'actor': 'Chad Larson'}],
 [{'reaction': 'ð\x9f\x98\x86', 'actor': 'Joanna Rusch'},
  {'reaction': 'ð\x9f\x98\x86', 'actor': 'Joanna Rusch'}],
 nan,
 [{'reaction': 'ð\x9f\x98\x83', 'actor': 'Chad Larson'},
  {'reaction': 'â\x9d¤', 'actor': 'Joanna Rusch'},
  {'reaction': 'ð\x9f\x98\x83', 'actor': 'Chad Larson'},
  {'reaction': 'â\x9d¤', 'actor': 'Joanna Rusch'}],
 [{'reaction': 'ð\x9f\x98\x83', 'actor': 'Chad Larson'},
  {'reaction': 'ð\x9f\

In [17]:
chessDf['reactions'].iloc[0][0]['reaction']

'ð\x9f\x91\x8d'

The code above is me trying to figure out which part of the reactions data I want to actually display in the collumn. I used .iloc to continue t odig into the reactions data until I found the actual reaction from the sender_name. I continued to check until the type switched from a list to a dict, because then I knew I would no longer be looking at a list of data, which would not fit/be tidy within a single collumn. This was the data that I wanted to replace the exsisting "reactions" collumn with. 

In [18]:
chessDf['reactions'] = chessDf['reactions'].fillna('')

In [19]:
chessDf['share'] = chessDf['share'].fillna('')

In [20]:
chessDf['photos'] = chessDf['photos'].fillna('')

In [21]:
chessDf['gifs'] = chessDf['gifs'].fillna('')

In [22]:
chessDf['users'] = chessDf['users'].fillna('')

The code from above was written by me while I was trying to figure out what to do about the "reactions" collumn. It replaces all of the na entries with a blank '' space. This is to ensure that the code meets tidy requirments. I found .fillna on the internet, and it was very helpful. 

In [23]:
chessDf['reactions'] = [[y['reaction'] for y in x] for x in chessDf['reactions']]

As I explained in my email, here is where I ran into trouble. I spent a while studying the code from the spotify in-class example and my notes from Thursday, then I began to write the code. Initally, I was having trouble defining the exact perimiters of what I wanted, but I was able to narrow it down to here. I cannot manage to get past this error, and I am not sure what exatly this means. I know what I want to do to fix this data and make it tidy, but I can't get past this step. I hope at least my intentions with the code are clear enough to get my point across. 

In [24]:
chessDf.head()

,sender_name,timestamp_ms,content,reactions,type,is_unsent,is_taken_down,bumped_message_metadata,share,photos,gifs,users
0,Joanna Rusch,1666374933946,Maybe he just wants to ride the publicity for ...,"[ð, ð]",Generic,False,False,{'bumped_message': 'Maybe he just wants to rid...,,,,
1,Chad Larson,1666373448613,To be fair to Hans....no one wants to be assoc...,"[ð, ð]",Generic,False,False,{'bumped_message': 'To be fair to Hans....no o...,,,,
2,Chad Larson,1666373216381,He would have to prove he didn't cheat and tha...,"[ð, ð]",Generic,False,False,{'bumped_message': 'He would have to prove he ...,,,,
3,Scott Pence,1666373164883,"Yeah, no way. You over shoot and hope to get a...","[ð, ð]",Generic,False,False,"{'bumped_message': 'Yeah, no way. You over sho...",,,,
4,Chad Larson,1666373111157,"From what I see, I don't think he could win. ...",[],Generic,False,False,"{'bumped_message': 'From what I see, I don't t...",,,,


Question 1: I have set the row-level to respresent a single message, while each column represents a single variable or attribute about the message. For example, the row labeled "0" is representative of a single message, and each following item in every column corresponds to the title of that column. Going from right to left: sender_name represents the name of the message sender. timestamp_ms respresents the timestamp that the message was sent. content is the actual message sent. reactions is any emojis or responses given to a message. type is the form of the message. is_taken_down checks if the message has been taken down. bumped_message_metadata is any metadata pertaing to the message. The following 3, share, photos, and gifs are all attachments that could be made to a message that was sent. 

Question 2: My data frame meets the three core requirments of being tidy data, although it is not exactly perfect. Each variable forms a column, each observation (in this case, a message), and only one observational unit is being tracked in this table. All 3 core requirments listed in the readings are being met, and some additional ones are as well. For example all slots that were originally Nan have been filled, even if only with blank " ", as this is also a trait of tidy data. There are not, as far as I can tell, multiple variables stored in a single column nor are there variables stored in both columns and rows. There are some things that I would like to do to the code that would make it more easily understood/legible. These fall outside of the requirments for tidy data, but they would certainly help in many ways. For one, I would consider giving more accurate/understandable names to come of the categories, such as users or type, as they aren't easy to understand on first glance. Secondly, I would maybe consider cutting down on or removing some of the things measured by this data, as it isn't all as relevant as it may seem. Overall, if we had to use the rating system we used in class (Good, Bad, Ugly), I would give this data an "ugly", but with a few tweaks it could push over into good.

Question 3: There are many things I would consider for a visualization that would let someone explore and understand with their data to grow a better understanding of it. One of the things I would do would be to a timeline or line graph to explore the messages sent over time based on the timestamp_ms variable. This would orginize each message by the time that it was sent and would help users follow the conversation better and see how some variables, like reactions, increased over time. In this line graph, most data would be left out, as I don't think that much of it is needed. The goal of my visualization would be to the frequency of messages over time, as opposed to the content or reaction to the messages. This is because I belive it is easier to understand numbers or data information from visualizations than it is to understand or consume content. In this situation, the x-axis would be time, and the y-axsis would be number of messages sent. Another thing I would consider would be mapping the share, photo, user, and gif variables to a hamburger menu or some other drop-down solution, as many if not most of the messages do not have any data in these variables, meaning they are taking up much needed space. This is only possible if the data is interactive, which isn't guarenteed, but if it is, then this would be a good call. I think there are a lot of ways you could manipulate the data visualization to show many different messages.